<a href="https://colab.research.google.com/github/harsha-9977/AIML/blob/main/web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi[all] jinja2 geopy folium pyngrok uvicorn --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 24.4 MB/s eta 0:00:00


In [2]:
import os
os.makedirs("templates", exist_ok=True)


In [3]:
%%writefile map_api.py
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from geopy.geocoders import Nominatim
import folium
import os

# App setup
app = FastAPI()
templates = Jinja2Templates(directory="templates")

# Create templates folder
os.makedirs("templates", exist_ok=True)

@app.get("/", response_class=HTMLResponse)
def form_get(request: Request):
    return templates.TemplateResponse("form.html", {"request": request})

@app.post("/", response_class=HTMLResponse)
def form_post(request: Request, city: str = Form(...)):
    geolocator = Nominatim(user_agent="map_app")
    location = geolocator.geocode(city)

    if not location:
        return templates.TemplateResponse("form.html", {"request": request, "error": "City not found"})

    lat, lon = location.latitude, location.longitude

    # Create Folium map
    m = folium.Map(location=[lat, lon], zoom_start=11)
    folium.Circle(
        location=[lat, lon],
        radius=25000,
        color="blue",
        fill=True,
        fill_opacity=0.3,
    ).add_to(m)

    map_path = "templates/map.html"
    m.save(map_path)

    return templates.TemplateResponse("map.html", {"request": request, "city": city})


Overwriting map_api.py


In [4]:
%%writefile templates/form.html
<!DOCTYPE html>
<html>
<head><title>City Map App</title></head>
<body>
  <h2>Enter City to View Map (50km x 50km)</h2>
  <form method="post">
    <input type="text" name="city" placeholder="e.g. Paris" required>
    <button type="submit">Show Map</button>
  </form>
  {% if error %}
    <p style="color:red;">{{ error }}</p>
  {% endif %}
</body>
</html>


Overwriting templates/form.html


In [5]:
%%writefile templates/map.html
<!DOCTYPE html>
<html>
<head><title>Map for {{ city }}</title></head>
<body>
  <h2>Map of {{ city }} (50km x 50km)</h2>
  {% include 'map.html' %}
</body>
</html>


Overwriting templates/map.html


In [6]:
!ngrok config add-authtoken 2zGQ8JjiFanrPui3ZAAfrPsEa4B_6fSgyfoSXiYTNoh5n41yQ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok
import threading
import time
import subprocess

# Stop any previous tunnel (just in case)
ngrok.kill()

# Step 1: Start FastAPI app using uvicorn in the background
def run_uvicorn():
    subprocess.Popen(["uvicorn", "map_api:app", "--host", "0.0.0.0", "--port", "8501"])

# Step 2: Run app thread
threading.Thread(target=run_uvicorn).start()

# Step 3: Wait for the server to be ready
time.sleep(5)  # Give Uvicorn enough time to start

# Step 4: Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🚀 App is live at: {public_url}")


🚀 App is live at: NgrokTunnel: "https://a23a-35-225-12-115.ngrok-free.app" -> "http://localhost:8501"


In [10]:
%%writefile requirements.txt
fastapi
uvicorn
jinja2
geopy
folium


Writing requirements.txt


In [12]:
%%writefile start.sh
#!/bin/bash
uvicorn map_api:app --host=0.0.0.0 --port=$PORT


Writing start.sh


In [14]:
!chmod +x start.sh


In [1]:
!zip -r web-app.zip map_api.py requirements.txt start.sh templates/


  adding: map_api.py (deflated 58%)
  adding: requirements.txt (stored 0%)
  adding: start.sh (deflated 5%)
  adding: templates/ (stored 0%)
  adding: templates/map.html (deflated 67%)
  adding: templates/form.html (deflated 32%)


In [2]:
from google.colab import files
files.download("web-app.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>